# Trait `DataStructurable`

In [1]:
# import classes
from dataclasses import dataclass
from python_middlewareable import (
    RequestBase,
    MiddlewareBase,
    MiddlewareNextCallBase,
    MiddlewareableBase,
    DataStructurableRequestMixin,
    DataStructurable,
    RequestDataBase,
    ResponseDataBase,
    TransportDataBase,
)


# define a payload structure
@dataclass
class RequestData(RequestDataBase):  # inherit from RequestDataBase
    name: str


# define a response structure
@dataclass
class ResponseData(ResponseDataBase):  # inherit from ResponseDataBase
    value: str = ""


# add the `DataStructurableRequestMixin` to the request class
@dataclass
class Request(
    DataStructurableRequestMixin[RequestData, ResponseData, TransportDataBase],
    RequestBase,
):
    pass


# define middlewares
class OneMiddleware(MiddlewareBase[Request]):
    async def handle(
        self, request: Request, next_call: MiddlewareNextCallBase[Request]
    ) -> Request:
        request.response_data.value = request.request_data.name

        request.response_data.value += " from OneMiddleware"

        result = await next_call(request)

        return result


class TwoMiddleware(MiddlewareBase[Request]):
    async def handle(
        self, request: Request, next_call: MiddlewareNextCallBase[Request]
    ) -> Request:
        request.response_data.value += " from TwoMiddleware"

        result = await next_call(request)

        return result


# add the trait mixin
class App(DataStructurable, MiddlewareableBase[Request]):
    middlewares = [OneMiddleware, TwoMiddleware]


# middlewareable
app = App()

# request
request = Request(
    request_data=RequestData(name="John"),
    response_data=ResponseData(),
    transport_data=TransportDataBase(),
)

# process request
result = await app.process_middlewares(request)

# check the result
print(result)

# output:
# Request(
#   request_data=RequestData(name='John'),
#   response_data=ResponseData(value='John from OneMiddleware from TwoMiddleware'),
#   transport_data=TransportDataBase(),
# )

Request(request_data=RequestData(name='John'), response_data=ResponseData(value='John from OneMiddleware from TwoMiddleware'), transport_data=<python_middlewareable.traits.data_structurable.dtos.transport_data.TransportData object at 0x7fc69c724c10>)
